# 第9回: RNN: Exercise2
## シェイクスピアをCharacter-Level RNNで学習する
### 目標
- ChainerでRNNを実装する
- 言語モデルを学習させる
- truncated backpropを実装する

### おまじない
必要なモジュールをimportしましょう。

In [1]:
%matplotlib inline
import chainer
from chainer import cuda, Function, optimizers
from chainer import Link, Chain, ChainList, Variable
import chainer.functions as F
import chainer.links as L
import matplotlib.pyplot as plt
import numpy as np
xp = np

## Character Level言語モデル

ある入力列$x_1, x_2, \dots, x_t$が与えられたときに、
$x_{t+1}$を予測するモデルを文字レベルの言語モデル(character level language model)と呼びます。すなわちこのモデルは、'hell'が与えられたときに、次に現れるアルファベット'o'を予測する事ができます。

本演習では、Shakespeareの悲劇の一つである『リア王』を用いて、Character Level Language Modelを学習させてみます。

## 学習データの構造と前処理
### 学習データの構造
さて、学習に用いるデータセットは、例えば以下のような文章で構成されています。

```
BIONDELLO:
Marry, that it may not pray their patience.'

KING LEAR:
The instant common maid, as we may less be
a brave gentleman and joiner: he that finds us with wax
And owe so full of presence and our fooder at our
staves. It is remorsed the bridal's man his grace
for every business in my tongue, but I was thinking
that he contends, he hath respected thee.
```

文章を実際に扱うには、幾つかのプログラム上の工夫が必要になります。
まず、学習データ中で使用される文字や記号をリカレントニューラルネットワークに入力する際に、これらの文字や記号をベクトルで表記する必要があります。例えば、'b'という文字をネットワークに入力する際には、アルファベットの二番目の文字なので、`...00010`というように、下から2bit目を立たせて表現します。

このように、記号をネットワークで扱いやすいような
分散表現に変換する操作のことをword embeddings(単語の分散表現)と呼びます。
実際に言語モデルや翻訳モデルを獲得する際には、
このように文字(character)を変換するのではなく、よりまとまった表現として語(word)を分散表現に変換する
Word to Vector(W2V)という手法も広く使われることも覚えておくと良いでしょう。

Chainerはこうした文字埋め込みの分散表現が`links.EmbedID`に実装されています。
例えば、`abc...xyz`(小文字アルファベット、
23文字)から構成される、任意の時系列データを学習させる場合の前処理は、
以下のように記述できます。

##### Example
```python
embed = L.EmbedID(23, l1_hidden_units)
```

### 前処理(preprocessing)

今回使用するデータセットは、アルファベット(大文字、小文字)、
及び句読記号(コンマ、コロン、セミコロンなど)にから構成されそうです。

実際にデータ・セットを読み込んで確かめてみましょう。

In [2]:
# load text file
with open('./data/RNN/shakespear.txt', 'r') as full_text:
    full_text = open('./data/RNN/shakespear.txt', 'r').readlines()

In [3]:
# check how it looks like (if you want)
# print(full_text)

さて、`list`に格納された`full_text`を学習データセットとして扱うには少し工夫が必要です。

すなわち、
- 段落の終わりを告げる`<EOP>`
- 行の終わりを告げる`<EOL>`

を導入する必要があります。

これによって、データをネットワークにfeedする際に区切りをつけて処理をすることが出来ます。

以上の前処理(preprocessing)は、以下のように実装されます。

In [4]:
symbol = {'<EOL>': 0, '<EOP>': 1}
dataset = []
for line in full_text:
    if line == '\n':
        dataset.append(Variable(xp.array([symbol['<EOP>']]).astype(np.int32)))
        continue
    for letter in line.replace('\n', ''):
        if not letter in symbol:
            symbol[letter] = len(symbol)
        dataset.append(Variable(xp.array([symbol[letter]]).astype(np.int32)))
    dataset.append(Variable(xp.array([symbol['<EOL>']]).astype(np.int32)))

In [5]:
# symbol dict: symbols -> id
print(symbol)

{'<EOL>': 0, '<EOP>': 1, 'T': 2, 'h': 3, 'a': 4, 't': 5, ',': 6, ' ': 7, 'p': 8, 'o': 9, 'r': 10, 'c': 11, 'n': 12, 'e': 13, 'm': 14, 'l': 15, 'i': 16, "'": 17, 'd': 18, 'u': 19, 's': 20, 'f': 21, 'I': 22, 'y': 23, 'v': 24, ';': 25, 'q': 26, 'H': 27, 'b': 28, 'k': 29, 'x': 30, 'A': 31, 'B': 32, 'w': 33, 'g': 34, 'G': 35, '.': 36, 'O': 37, 'N': 38, 'D': 39, 'E': 40, 'L': 41, ':': 42, 'M': 43, 'K': 44, 'R': 45, 'j': 46, 'S': 47, '-': 48, 'F': 49, 'W': 50, 'U': 51, 'Y': 52, 'C': 53, 'V': 54, 'P': 55, '?': 56, '!': 57, 'J': 58, 'X': 59, 'z': 60, 'Q': 61, 'Z': 62}


In [6]:
# inv dict: id -> symbols
symbol_inv = {v: k for k, v in symbol.items()}
print(symbol_inv)

{0: '<EOL>', 1: '<EOP>', 2: 'T', 3: 'h', 4: 'a', 5: 't', 6: ',', 7: ' ', 8: 'p', 9: 'o', 10: 'r', 11: 'c', 12: 'n', 13: 'e', 14: 'm', 15: 'l', 16: 'i', 17: "'", 18: 'd', 19: 'u', 20: 's', 21: 'f', 22: 'I', 23: 'y', 24: 'v', 25: ';', 26: 'q', 27: 'H', 28: 'b', 29: 'k', 30: 'x', 31: 'A', 32: 'B', 33: 'w', 34: 'g', 35: 'G', 36: '.', 37: 'O', 38: 'N', 39: 'D', 40: 'E', 41: 'L', 42: ':', 43: 'M', 44: 'K', 45: 'R', 46: 'j', 47: 'S', 48: '-', 49: 'F', 50: 'W', 51: 'U', 52: 'Y', 53: 'C', 54: 'V', 55: 'P', 56: '?', 57: '!', 58: 'J', 59: 'X', 60: 'z', 61: 'Q', 62: 'Z'}


In [8]:
print([v.data[0] for v in dataset])

[2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 7, 11, 9, 12, 5, 13, 14, 8, 5, 6, 7, 9, 10, 7, 11, 15, 4, 16, 14, 17, 18, 7, 5, 3, 9, 19, 7, 20, 15, 13, 8, 5, 7, 20, 9, 7, 21, 4, 16, 5, 3, 21, 19, 15, 6, 0, 22, 7, 14, 4, 23, 7, 11, 9, 12, 5, 10, 16, 24, 13, 7, 9, 19, 10, 7, 21, 4, 5, 3, 13, 10, 25, 7, 4, 12, 18, 6, 7, 16, 12, 7, 5, 3, 13, 16, 10, 7, 18, 13, 21, 13, 4, 5, 13, 18, 7, 26, 19, 13, 13, 12, 6, 0, 27, 13, 10, 7, 21, 15, 13, 20, 3, 7, 28, 10, 9, 29, 13, 7, 14, 13, 7, 4, 12, 18, 7, 8, 19, 5, 5, 4, 12, 11, 13, 7, 9, 21, 7, 13, 30, 8, 13, 18, 16, 5, 16, 9, 12, 7, 3, 9, 19, 20, 13, 6, 0, 31, 12, 18, 7, 16, 12, 7, 5, 3, 4, 5, 7, 20, 4, 14, 13, 7, 5, 3, 4, 5, 7, 13, 24, 13, 10, 7, 22, 7, 15, 4, 14, 13, 12, 5, 7, 5, 3, 16, 20, 7, 20, 5, 9, 14, 4, 11, 3, 6, 0, 31, 12, 18, 7, 3, 13, 6, 7, 12, 9, 10, 7, 32, 19, 5, 15, 23, 7, 4, 12, 18, 7, 14, 23, 7, 21, 19, 10, 23, 6, 7, 29, 12, 9, 33, 16, 12, 34, 7, 13, 24, 13, 10, 23, 5, 3, 16, 12, 34, 0, 35, 10, 13, 33, 7, 18, 4, 16, 15, 23, 7, 13, 24, 13, 10, 6, 7,

上記に対応する`symbol`は以下のようになります。

In [9]:
print([symbol_inv[v.data[0]] for v in dataset][0:400])

['T', 'h', 'a', 't', ',', ' ', 'p', 'o', 'o', 'r', ' ', 'c', 'o', 'n', 't', 'e', 'm', 'p', 't', ',', ' ', 'o', 'r', ' ', 'c', 'l', 'a', 'i', 'm', "'", 'd', ' ', 't', 'h', 'o', 'u', ' ', 's', 'l', 'e', 'p', 't', ' ', 's', 'o', ' ', 'f', 'a', 'i', 't', 'h', 'f', 'u', 'l', ',', '<EOL>', 'I', ' ', 'm', 'a', 'y', ' ', 'c', 'o', 'n', 't', 'r', 'i', 'v', 'e', ' ', 'o', 'u', 'r', ' ', 'f', 'a', 't', 'h', 'e', 'r', ';', ' ', 'a', 'n', 'd', ',', ' ', 'i', 'n', ' ', 't', 'h', 'e', 'i', 'r', ' ', 'd', 'e', 'f', 'e', 'a', 't', 'e', 'd', ' ', 'q', 'u', 'e', 'e', 'n', ',', '<EOL>', 'H', 'e', 'r', ' ', 'f', 'l', 'e', 's', 'h', ' ', 'b', 'r', 'o', 'k', 'e', ' ', 'm', 'e', ' ', 'a', 'n', 'd', ' ', 'p', 'u', 't', 't', 'a', 'n', 'c', 'e', ' ', 'o', 'f', ' ', 'e', 'x', 'p', 'e', 'd', 'i', 't', 'i', 'o', 'n', ' ', 'h', 'o', 'u', 's', 'e', ',', '<EOL>', 'A', 'n', 'd', ' ', 'i', 'n', ' ', 't', 'h', 'a', 't', ' ', 's', 'a', 'm', 'e', ' ', 't', 'h', 'a', 't', ' ', 'e', 'v', 'e', 'r', ' ', 'I', ' ', 'l', 'a', 'm

<div class="alert alert-block alert-info">
Note:<br>
以上がネットワークにどのような情報を与えるのか決める、
前処理(preprocessing)の作業となります。
深層学習による画像処理ではend-to-endでの学習が謳われていますが、
言語処理においてどのように前処理を行うのかという問題は、
望ましい出力を得るために重要となります。
</div>

## Character Level言語モデルを学習するRNNの実装
では、実際にCharacter Level Language Modelを学習するRNNを
`Chainer`で実装していきましょう。

In [10]:
class CLRNN(Chain):
    def __init__(
        self,
        n_input,
        n_hidden,
        n_output,
    ):
        super(CLRNN, self).__init__()
        with self.init_scope():
            self.input = L.EmbedID(n_input, n_hidden)
            self.lstm = L.LSTM(n_hidden, n_hidden)
            self.out = L.Linear(n_hidden, n_output)
            
    def __call__(self, x):
        x = self.input(x)
        h = self.lstm(x)
        y = self.out(h)
        return y
    
    def reset_state(self):
        '''
        a function to clear the states in the hidden layer
        '''
        self.lstm.reset_state()

## Unchainingの実装
### BPTTの問題
非常に長いシークエンスを学習することは、
勾配計算をかなり前まで遡らなければならないことを意味します。
こうした勾配計算は講義で解説したBPTT(Back Propagation Through Time)
として実行されますが、

* 計算過程で勾配消失・爆発を引き起こす
* (長期にわたる勾配計算では誤差計算を保持しなければならないため)メモリリソースを圧迫する

という点で問題があります。
これらの問題を解決するために、
しばしばバックプロパゲーションを短い時間範囲で切り捨てる場合があります。
この手法はtruncated backpropagationと呼ばれます。
これはヒューリスティックな手法であり、もちろん切り捨てられた勾配は失われますが、
上記のような問題を上手く解決し、結果的に学習効率を向上させることが出来ます。

### truncated backpropagation

`Chainer`にはtruncated backpropagationの実装として、
`Variable.unchain_backward()`というメソッドが実装されています。
`Backward Unchaining`は変数から計算履歴(勾配計算)の履歴を断ち切ります。

`truncated backpropagation`の例を以下に示します。

In [12]:
rnn = CLRNN(n_input=len(symbol), n_hidden=1000, n_output=len(symbol))
model = L.Classifier(rnn)
optimizer = optimizers.SGD()
optimizer.setup(model)

In [13]:
# train rnn with truncated backpropagation
loss = 0
count = 0
seqlen = len(dataset)

rnn.reset_state()
for cur_word, next_word in zip(dataset, dataset[1:]):
    loss += model(cur_word, next_word)
    count += 1
    if count % 30 == 0 or count == seqlen:
        print(loss.data)
        model.cleargrads()
        loss.backward()
        loss.unchain_backward()
        optimizer.update()

122.68234252929688
240.45733642578125
341.25372314453125
445.0404968261719
561.2396240234375
660.0677490234375
757.9274291992188
855.3002319335938
966.7550659179688
1061.722412109375
1158.4852294921875
1277.5400390625
1353.2667236328125
1467.2490234375
1559.4169921875
1645.710693359375
1743.203369140625
1828.093017578125
1915.887939453125
2004.55078125
2086.94921875
2178.467529296875
2260.13232421875
2351.91064453125
2437.079833984375
2529.2109375
2604.090576171875
2690.905029296875
2783.835205078125
2868.443603515625
2951.359375
3046.84130859375


KeyboardInterrupt: 

各ステップでの誤差は`loss`に蓄積されています。
上記実装では、30stepごとに蓄積された誤差に対してバックプロパゲーションを行うと同時に、
`loss.unchain_backward()`メソッドを呼び出すことで、
蓄積された損失を計算履歴から消去(切り捨て)しています。

## 順伝搬の計算
### 計算履歴を保存しないネットワーク評価
RNNの順伝搬計算で必要なのは、入力及び一時刻前の隠れ層の状態のみで、
逆伝搬のように計算履歴をすべて保存する必要はありません。
`Chainer`では計算履歴を保存しないForward用の`chainer.config.enable_backprop`という`flag`を用意しています。(Chainer v1で使用されていた`volatile option`はv2で[廃止されました]((http://docs.chainer.org/en/stable/upgrade.html?highlight=volatile#volatile-flag-is-removed))

##### Example
```python
with chainer.no_backprop_mode():
    x = Variable(x_data)
    feat = fixed_func(x)
y = predictor_func(feat)
y.backward()

```

<div class="alert alert-block alert-warning">
Warning:<br>
no_backprop_modeを使用した場合、loss.backward()は呼び出せなくなります。
</div>

In [ ]:
rnn = RNN()
model = L.Classifier(rnn)
optimizer = optimizers.SGD()
optimizer.setup(model)

# train rnn with truncated backpropagation
loss = 0
count = 0
seqlen = len(dataset)

rnn.reset_state()
for cur_word, next_word in zip(dataset, dataset[1:]):
    loss += model(cur_word, next_word)
    count += 1
    if count % 30 == 0 or count == seqlen:
        model.cleargrads()
        loss.backward()
        loss.unchain_backward()
        optimizer.update()